feature selection techniques

including ANOVA, Chi-Squared, Forward Selection, Recursive Feature Elimination (RFE), Random Forestbased selection, and Decision TREE 

# import and setup


 input variables 

In [2]:
# Define base path
base_path1 = "/Users/priyam/paper_recreation/UCI_HAR_FEATURE_ANALYSIS_MODELLING/stage1_Custom Feature_Extraction_ModelTraining/DATA_EXTRACTED_WITH_99_FEATURES"

# Load clean data
X = pd.read_csv(os.path.join(base_path1, "X_CLEAN.csv"))
y = pd.read_csv(os.path.join(base_path1, "Y_CLEAN.csv")).values.ravel()

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


 defining the models 

# model evaluation and training 

#### FETAURE SELCTION method list

In [ ]:
# === FEATURE SELECTION METHODS ===
feature_methods = {
    "ANOVA": SelectKBest(score_func=f_classif, k=40),
    "ChiSquared": SelectKBest(score_func=chi2, k=40),
    "RFE": RFE(estimator=LogisticRegression(max_iter=1000), n_features_to_select=40),
    "ForwardSelection": SequentialFeatureSelector(estimator=LogisticRegression(max_iter=500), n_features_to_select=40, direction='forward'),
    "RandomForestImportance": RandomForestClassifier(),
    "DecisionTreeImportance": DecisionTreeClassifier()
}


 MODEL EVALUATION funcn

In [6]:
# === EVALUATION FUNCTION ===
def evaluate_model(y_test, y_pred):
    return {
        "Accuracy": accuracy_score(y_test, y_pred),
        "F1": f1_score(y_test, y_pred, average='weighted'),
        "Precision": precision_score(y_test, y_pred, average='weighted'),
        "Recall": recall_score(y_test, y_pred, average='weighted')
    }


#### main loop

In [7]:
feature_names = X.columns.tolist() if isinstance(X, pd.DataFrame) else [f"Feature {i}" for i in range(X.shape[1])]
import matplotlib.pyplot as plt

In [8]:
base_path= "/Users/priyam/paper_recreation/UCI_HAR_FEATURE_ANALYSIS_MODELLING/stage3_FeatureSelection_ModelEvaluation/CUSTOM_DATASET/COMPARISON_TABLES_40_FEATURES"
if not os.path.exists(base_path):
    os.makedirs(base_path)

In [ ]:
# main loop
feature_names = X.columns.tolist() if isinstance(X, pd.DataFrame) else [f"Feature {i}" for i in range(X.shape[1])]
import matplotlib.pyplot as plt
from tabulate import tabulate

for method_name, selector in feature_methods.items():
    print(f"\n🔍 Feature Selection: {method_name}")

    # Normalize for Chi-Squared
    if method_name == "ChiSquared":
        X_scaled = MinMaxScaler().fit_transform(X)
    else:
        X_scaled = X.copy()

    # Feature selection
    if method_name in ["RandomForestImportance", "DecisionTreeImportance"]:
        selector.fit(X_scaled, y)
        importances = selector.feature_importances_
        top_k_indices = np.argsort(importances)[-40:]
        selected_features = top_k_indices.tolist()
        # Convert to NumPy if DataFrame
        if isinstance(X_scaled, pd.DataFrame):
            X_selected = X_scaled.iloc[:, top_k_indices].values
        else:
            X_selected = X_scaled[:, top_k_indices]

        # Plot
        plt.figure(figsize=(8, 4))
        plt.barh(range(22), importances[top_k_indices])
        plt.yticks(ticks=range(22), labels=np.array(feature_names)[top_k_indices])
        plt.xlabel("Importance")
        plt.title(f"{method_name} - Top 40 Features")
        plot_path = os.path.join(base_path, f"{method_name}_RESULTS", f"{method_name}_feature_importance.png")
        os.makedirs(os.path.dirname(plot_path), exist_ok=True)
        plt.tight_layout()
        plt.savefig(plot_path)
        plt.close()
    
    else:
        selector.fit(X_scaled, y)
        mask = selector.get_support()
        selected_features = np.where(mask)[0].tolist()
        # Convert to NumPy if DataFrame
        if isinstance(X_scaled, pd.DataFrame):
            X_selected = X_scaled.iloc[:, mask].values
        else:
            X_selected = X_scaled[:, mask]
            
        
    # Save selected features
    selected_feature_names = [feature_names[i] for i in selected_features]
    output_folder = os.path.join(base_path, f"{method_name}_RESULTS")
    os.makedirs(output_folder, exist_ok=True)
    
    with open(os.path.join(output_folder, f"{method_name}_selected_features.txt"), 'w') as f:
        for name in selected_feature_names:
            f.write(name + '\n')

    # train and evaluate models with selected features
    X_train, X_test, y_train, y_test = train_test_split(X_selected, y, test_size=0.2, random_state=42)

    results = []
    for model_name, model in models.items():
        try:
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            scores = evaluate_model(y_test, y_pred)
            results.append([model_name, scores["Accuracy"], scores["F1"], scores["Precision"], scores["Recall"]])
        except Exception as e:
            print(f" Error in {model_name}: {e}")
            results.append([model_name, "ERR", "ERR", "ERR", "ERR"])
    # Save results
    output_folder = os.path.join(base_path, f"{method_name}_RESULTS")
    os.makedirs(output_folder, exist_ok=True)
    df_results = pd.DataFrame(results, columns=["Model", "Accuracy", "F1 Score", "Precision", "Recall"])
    output_txt_path = os.path.join(output_folder, f"{method_name}_model_scores.txt")
    
    table_string = tabulate(df_results, headers='keys', tablefmt='grid', showindex=False)
    
    with open(output_txt_path, 'w') as f:
        f.write(table_string)
    
    print(f"✔ Results saved to: {output_txt_path}")


# TOP 20 FEATURE PLOTS

In [ ]:
base_path2= "/Users/priyam/paper_recreation/UCI_HAR_FEATURE_ANALYSIS_MODELLING/stage3_FeatureSelection_ModelEvaluation/CUSTOM_DATASET/TOP_20_FEATURE_PLOT"
os.makedirs(base_path2, exist_ok=True)

In [ ]:

y = np.array(y_test).ravel()     # Ensure y is a 1D array
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_test)
feature_names = X.columns if isinstance(X, pd.DataFrame) else [f"f{i}" for i in range(X.shape[1])]


In [ ]:
scores = {}

# ANOVA F-score
anova_selector = feature_methods["ANOVA"].fit(X_scaled, y)
scores["F-score (ANOVA)"] = anova_selector.scores_

# Chi2
chi2_selector = feature_methods["ChiSquared"].fit(X_scaled, y)
scores["Chi2 score"] = chi2_selector.scores_

# RFE
rfe_selector = feature_methods["RFE"].fit(X_scaled, y)
scores["RFE ranking"] = -rfe_selector.ranking_  # lower is better → invert for plotting

# Forward Selection
fs_selector = feature_methods["ForwardSelection"].fit(X_scaled, y)
scores["Forward selection"] = fs_selector.get_support().astype(int)  # 1 if selected

# Random Forest Importance
rf_model = feature_methods["RandomForestImportance"].fit(X_scaled, y)
scores["RandomForest Importance"] = rf_model.feature_importances_

# Decision Tree Importance
dt_model = feature_methods["DecisionTreeImportance"].fit(X_scaled, y)
scores["DecisionTree Importance"] = dt_model.feature_importances_


score_df = pd.DataFrame(scores, index=feature_names)

for score_type in score_df.columns:
    top_20 = score_df[score_type].nlargest(20)
    plt.figure(figsize=(10, 6))
    top_20.plot(kind='barh')
    plt.gca().invert_yaxis()
    plt.title(f"Top 20 Features by {score_type}")
    plt.xlabel("Score")
    plt.tight_layout()
    #plt.show()
    
    plot_path = os.path.join(base_path2, f"{score_type}_top_20_features.png")
    plt.savefig(plot_path)
    plt.close()
    print(f"✔ Plot saved to: {plot_path}")
    # Save the DataFrame to a CSV file




# deciding k for each techniques 


In [ ]:
base_path4= "/Users/priyam/paper_recreation/HAR MODEL_OPTIMIZATION/stage3_FeatureSelection_ModelEvaluation/CUSTOM_DATASET/OUTPUT_CD"

In [ ]:
# === FEATURE SELECTION METHODS ===
feature_methods = {
    "ANOVA": SelectKBest(score_func=f_classif, k=20),
    "ChiSquared": SelectKBest(score_func=chi2, k=28),
    "RFE": RFE(estimator=LogisticRegression(max_iter=5000), n_features_to_select=10),
    "ForwardSelection": SequentialFeatureSelector(estimator=LogisticRegression(max_iter=5000), n_features_to_select=10, direction='forward'),
    "RandomForestImportance": RandomForestClassifier(),
    "DecisionTreeImportance": DecisionTreeClassifier()
}


In [ ]:
from sklearn.feature_selection import f_classif, chi2
import matplotlib.pyplot as plt

for method_name, selector in feature_methods.items():
    print(f"📊 Feature Score Visualization: {method_name}")
    
    if method_name == "ChiSquared":
        X_scaled = MinMaxScaler().fit_transform(X)
    else:
        X_scaled = X.copy()
        
    if method_name in ["RandomForestImportance", "DecisionTreeImportance"]:
        selector.fit(X_scaled, y)
        scores = selector.feature_importances_

    elif method_name == "ANOVA":
        scores, _ = f_classif(X_scaled, y)

    elif method_name == "ChiSquared":
        scores, _ = chi2(X_scaled, y)

    elif method_name == "RFE":
        selector.fit(X_scaled, y)
        scores = -selector.ranking_  # lower is better, so invert for plotting

    elif method_name == "ForwardSelection":
        selector.fit(X_scaled, y)
        scores = selector.get_support().astype(int)  # 1 for selected, 0 otherwise

    else:
        continue
    # index as x axi  Plotting
    plt.figure(figsize=(10, 4))
    plt.bar(range(X.shape[1]), scores)
    plt.xlabel("Feature Index")
    plt.ylabel("Score / Importance")
    plt.title(f"{method_name}: Feature Ranking Scores")
    plt.tight_layout()
    #plt.show()

    # Save plot
    output_folder = os.path.join(base_path4, f"{method_name}_RESULTS")
    os.makedirs(output_folder, exist_ok=True)
    # Save the plot
    plot_path = os.path.join(output_folder, f"{method_name}_feature_scores.png")
    plt.savefig(plot_path)
    print(f" Plot saved to: {plot_path}")
    plt.close()  # Close the plot to free memory
    # Save feature names and scores


'''
    # feature nmaes as x axis 
    if isinstance(X, pd.DataFrame):
        feature_names = np.array(X.columns)
    else:
        feature_names = np.array([f"Feature {i}" for i in range(X.shape[1])])

    sorted_indices = np.argsort(scores)[::-1]
    sorted_scores = scores[sorted_indices]
    sorted_feature_names = feature_names[sorted_indices]

    plt.figure(figsize=(10, 4))
    plt.bar(sorted_feature_names, sorted_scores)
    plt.xticks(rotation=45, ha='right')
    plt.xlabel("Feature Name")
    plt.ylabel("Score / Importance")
    plt.title(f"{method_name} Feature Scores")
    plt.tight_layout()
    plt.show()
'''

In [ ]:
!pip install kneed

In [ ]:
from kneed import KneeLocator

suggested_k = {}

for method_name, selector in feature_methods.items():
    print(f"📊 Feature Score Analysis: {method_name}")
    
    if method_name == "ChiSquared":
        X_scaled = MinMaxScaler().fit_transform(X)
    else:
        X_scaled = X.copy()

    if method_name in ["RandomForestImportance", "DecisionTreeImportance"]:
        selector.fit(X_scaled, y)
        scores = selector.feature_importances_

    elif method_name == "ANOVA":
        scores, _ = f_classif(X_scaled, y)

    elif method_name == "ChiSquared":
        scores, _ = chi2(X_scaled, y)

    elif method_name == "RFE":
        selector.fit(X_scaled, y)
        scores = -selector.ranking_  # Invert so higher is better

    elif method_name == "ForwardSelection":
        selector.fit(X_scaled, y)
        scores = selector.get_support().astype(int)  # Not useful for ranking
        suggested_k[method_name] = np.sum(scores)
        continue  # skip plotting for binary mask

    else:
        continue

    sorted_scores = sorted(scores, reverse=True)
    kl = KneeLocator(range(1, len(sorted_scores)+1), sorted_scores, curve="convex", direction="decreasing")
    k = kl.knee if kl.knee is not None else 10  # fallback if knee not found
    suggested_k[method_name] = k

    # Plot
    plt.figure(figsize=(8, 4))
    plt.plot(range(1, len(sorted_scores)+1), sorted_scores, marker='o')
    if kl.knee:
        plt.axvline(k, color='red', linestyle='--', label=f'k = {k}')
    plt.title(f"{method_name} - Score Curve with Elbow")
    plt.xlabel("Feature Rank")
    plt.ylabel("Score / Importance")
    plt.legend()
    plt.tight_layout()
    #plt.show()
    # Save plot
    output_folder = os.path.join(base_path4, f"{method_name}_RESULTS")
    os.makedirs(output_folder, exist_ok=True)
    plot_path = os.path.join(output_folder, f"{method_name}_elbow_plot.png")
    plt.savefig(plot_path)
    print(f" Elbow plot saved to: {plot_path}")
    plt.close()  # Close the plot to free memory
print("\n📌 Suggested `k` values:")
for method, k in suggested_k.items():
    print(f"  {method}: k = {k}")
# Save suggested k values
suggested_k_path = os.path.join(base_path4, "suggested_k_values.csv")
suggested_k_df = pd.DataFrame(list(suggested_k.items()), columns=["Method", "Suggested k"])
suggested_k_df.to_csv(suggested_k_path, index=False)
print(f"Suggested k values saved to: {suggested_k_path}")
# Save feature names and scores


### 🧠 **What is `kneed` doing?**

**`kneed`** uses the **elbow method** to find the point where a curve (like feature importance or score) **starts to flatten** — this point is called the **"knee"** or **"elbow"**.

### 🔍 Why?

In feature selection, plotting scores sorted in descending order often shows:

* Sharp drops for important features
* A plateau for less informative features

The **elbow point** is a natural choice for `k`, the number of important features to keep.

---

### 📉 How it works internally:

* It looks at the curve formed by feature scores.
* Fits a **convex/concave curve**.
* Finds the point where the **slope changes sharply** — this is the "knee".

So `kneed` helps find this point **programmatically**, instead of visually inspecting every time.

---

## 📈 **What is this "score"?** (No heading shown in your plots)

It depends on the method:

| Feature Method       | "Score" Meaning                                             |
| -------------------- | ----------------------------------------------------------- |
| **ANOVA**            | F-statistic (higher = feature better separates classes)     |
| **ChiSquared**       | Chi-squared test statistic (measures dependency with label) |
| **RandomForest/DT**  | Gini-based or entropy-based importance                      |
| **RFE**              | Feature rank (lower = better), inverted here for plotting   |
| **ForwardSelection** | Binary mask (1 = selected, but no ranking available)        |

So this **score axis** on your plot is:

> How much a feature contributes to predicting the target — as defined by each selection method.

---

### 🎯 Summary:

* `kneed` uses **curve shape** to find where **score importance levels off**.
* The **"score"** depends on statistical strength or model-based importance.
* This helps pick a smart `k` without arbitrary guessing.




The "score" represents how important or useful each feature is for predicting the target variable — but the exact meaning depends on the method used:

* **In ANOVA (f\_classif)**:
  The score is the **F-statistic**, which compares the variance **between groups (classes)** to the variance **within groups**. A high F-value means the feature separates classes well — it's strongly correlated with the output.

* **In Chi-Squared test (chi2)**:
  The score is the **Chi-squared statistic**, which tests **independence** between the feature and the target. A high score means the feature and the target are likely **dependent** (i.e., the feature gives useful info about the target).

* **In Random Forest / Decision Tree**:
  The score is based on **feature importance**, calculated from how much a feature **reduces impurity (like Gini or entropy)** across all trees. Features that help split the data cleanly (low impurity) get higher importance scores.

* **In RFE (Recursive Feature Elimination)**:
  This method doesn't give scores directly but ranks features by how important they are to a model (e.g., logistic regression). We convert ranks into scores by taking the **negative rank**, so better-ranked features have higher scores.

* **In Forward Selection**:
  There’s no direct score; it selects a subset of features step-by-step based on which one improves the model most at each step. The result is a binary yes/no for each feature (selected or not), not a numeric score.

In all cases, higher scores mean the feature is more useful or informative for the prediction task.
